# Datsets UT Loading

Notebook to load Datsets ut data into the databse

## Imports

In [81]:
import psycopg2
import numpy as np
#import a folder in the parent directory
import sys
sys.path.append('../')
import dbtools.dbtools as qrs
from pathlib import Path
import tifffile as tiff
import pandas as pd
import os

## Connection

In [82]:
try:
    # Connect to the PostgreSQL database
    conn = qrs.connect()
    print("Connected to the database")

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

Connected to the database


## File loading

We load the dataset to check the path is correct and to extract some info

In [83]:
file_path = Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\04_ML_data\Juan Ignacio\JI_12\MonoElement\patch_vs_volfrac_9.csv')

df = pd.read_csv(file_path)

df.head(5)

,ut_rf_0,ut_rf_1,ut_rf_2,ut_rf_3,ut_rf_4,ut_rf_5,ut_rf_6,ut_rf_7,ut_rf_8,ut_rf_9,...,ut_rf_40897,ut_rf_40898,ut_rf_40899,ut_rf_40900,ut_rf_40901,ut_rf_40902,ut_rf_40903,ut_rf_40904,volfrac,areafrac
0,128.0,129.0,128.0,129.0,129.0,128.0,128.0,129.0,129.0,128.0,...,129.0,128.0,129.0,128.0,129.0,128.0,129.0,129.0,0.001043,-6.071332
1,128.0,129.0,129.0,128.0,129.0,128.0,129.0,128.0,128.0,129.0,...,129.0,128.0,129.0,128.0,129.0,129.0,128.0,129.0,0.001253,-7.247962
2,128.0,129.0,128.0,129.0,128.0,129.0,129.0,128.0,128.0,129.0,...,129.0,128.0,129.0,129.0,128.0,129.0,128.0,128.0,0.001710,-9.410326
3,129.0,128.0,129.0,128.0,128.0,129.0,129.0,128.0,129.0,128.0,...,128.0,128.0,129.0,129.0,128.0,129.0,128.0,128.0,0.001899,-10.423234
4,128.0,129.0,129.0,128.0,128.0,129.0,128.0,129.0,129.0,128.0,...,129.0,129.0,128.0,128.0,129.0,128.0,129.0,129.0,0.001886,-10.300951


## Datset data

Now we have to set the attributes and metadata of the Datsets:

1. Main attributes

    1. file_paths: Its a list with the paths of all the present files in the dataset. If the paths are not written as in the database it wont work so be precise.

    2. file_path: The dataset location

2. Metadata

    1. rows: Number of instances in the dataset

    2. shape: Shape to reconstruct the images from the dataset.

    3. patch_size: Size of the patch

    4. target: Target of the dataset, may be Volfrac, Areafrac, or any xct extracted feature. Multiple targets can be in the same dataset.
    

In [84]:
file_paths = [Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\12\frontal_90right.tif'),Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\03_UT_data\Probetas JI\probetas\12\12.tif')]

main_parameters = {'file_path':file_path}

description = 'Monoelement RF vs volfrac and areafrac. First functional group of datasets from 2024'

main_parameters['description'] = description

# metadata

#for each parameter of the Datset a metadata has to be created

metadata_parameters = []

#rows
metadata_parameters.append({'key':'rows', 'value':len(df), 'type':'cardinal'})

#shape
# metadata_parameters.append({'key':'shape', 'value':, 'type':'cardinal'})

#patch_size
metadata_parameters.append({'key':'patch_size', 'value':9, 'type':'pixels'})

#target
metadata_parameters.append({'key':'target', 'value':'volfrac', 'type':'nominal'})

#target
metadata_parameters.append({'key':'target2', 'value':'areafrac', 'type':'nominal'})

## Load into the table

In [85]:
print('Parameters to be inserted: ')
for key, value in main_parameters.items():
    print(f"-    {key}: {value}")

Parameters to be inserted: 
-    file_path: \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\04_ML_data\Juan Ignacio\JI_12\MonoElement\patch_vs_volfrac_9.csv
-    description: Monoelement RF vs volfrac and areafrac. First functional group of datasets from 2024


In [86]:
table_name = 'datasets'

# Extract column names and values from the attributes dictionary
columns = ', '.join(main_parameters.keys())
values = ', '.join([f"'{v}'" for v in main_parameters.values()])

# Construct the SQL INSERT statement
sql = f"INSERT INTO {table_name} ({columns}) VALUES ({values})"

print(sql)

# Create a cursor object using the cursor() method
cursor = conn.cursor()

# Execute the SQL statement
cursor.execute(sql)

cursor.execute('COMMIT')

cursor.close()

INSERT INTO datasets (file_path, description) VALUES ('\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\04_ML_data\Juan Ignacio\JI_12\MonoElement\patch_vs_volfrac_9.csv', 'Monoelement RF vs volfrac and areafrac. First functional group of datasets from 2024')


## Make sure insert was correct

In [87]:
data = qrs.get_data_metadata(table_name)

data

,id_dataset,file_path_dataset,description_dataset,patch_size_dataset,target_dataset,target2_dataset,rows_dataset,shape_dataset
0,1,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,Monoelement RF vs volfrac and areafrac. First ...,3 pixels,volfrac nominal,areafrac nominal,2686 cardinal,NaN
1,2,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,Monoelement RF vs volfrac and areafrac. First ...,3 pixels,volfrac nominal,areafrac nominal,2686 cardinal,NaN
2,3,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,Monoelement RF vs volfrac and areafrac. First ...,3 pixels,volfrac nominal,areafrac nominal,2923 cardinal,NaN
3,4,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,Monoelement RF vs volfrac and areafrac. First ...,3 pixels,volfrac nominal,areafrac nominal,2812 cardinal,NaN
4,5,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,Monoelement RF vs volfrac and areafrac. First ...,3 pixels,volfrac nominal,areafrac nominal,2584 cardinal,NaN
5,6,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,Monoelement RF vs volfrac and areafrac. First ...,3 pixels,volfrac nominal,areafrac nominal,2923 cardinal,NaN
6,7,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,Monoelement RF vs volfrac and areafrac. First ...,3 pixels,volfrac nominal,areafrac nominal,2584 cardinal,NaN
7,8,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,Monoelement RF vs volfrac and areafrac. First ...,5 pixels,volfrac nominal,areafrac nominal,2356 cardinal,NaN
8,9,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,Monoelement RF vs volfrac and areafrac. First ...,5 pixels,volfrac nominal,areafrac nominal,2356 cardinal,NaN
9,10,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,Monoelement RF vs volfrac and areafrac. First ...,5 pixels,volfrac nominal,areafrac nominal,2356 cardinal,NaN


In [88]:
row_id = data['id_dataset'].values[-1]

print(f"Row ID: {row_id}")

Row ID: 64


## Load into the metadata table

In [89]:
metadata_table_name =  table_name[:-1] + '_metadata'

for attributes in metadata_parameters:

    attributes[table_name[:-1] + '_id'] = row_id

    # Extract column names and values from the attributes dictionary
    columns = ', '.join(attributes.keys())
    values = ', '.join([f"'{v}'" for v in attributes.values()])

    # Construct the SQL INSERT statement
    sql = f"INSERT INTO {metadata_table_name} ({columns}) VALUES ({values})"

    print(sql)

    # Create a cursor object using the cursor() method
    cursor = conn.cursor()

    # Execute the SQL statement
    cursor.execute(sql)

    cursor.execute('COMMIT')

    cursor.close()

INSERT INTO dataset_metadata (key, value, type, dataset_id) VALUES ('rows', '1792', 'cardinal', '64')
INSERT INTO dataset_metadata (key, value, type, dataset_id) VALUES ('patch_size', '9', 'pixels', '64')
INSERT INTO dataset_metadata (key, value, type, dataset_id) VALUES ('target', 'volfrac', 'nominal', '64')
INSERT INTO dataset_metadata (key, value, type, dataset_id) VALUES ('target2', 'areafrac', 'nominal', '64')


## Make sure insert was correct

In [90]:
data = qrs.get_data_metadata(table_name)

data

,id_dataset,file_path_dataset,description_dataset,patch_size_dataset,target_dataset,target2_dataset,rows_dataset,shape_dataset
0,1,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,Monoelement RF vs volfrac and areafrac. First ...,3 pixels,volfrac nominal,areafrac nominal,2686 cardinal,NaN
1,2,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,Monoelement RF vs volfrac and areafrac. First ...,3 pixels,volfrac nominal,areafrac nominal,2686 cardinal,NaN
2,3,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,Monoelement RF vs volfrac and areafrac. First ...,3 pixels,volfrac nominal,areafrac nominal,2923 cardinal,NaN
3,4,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,Monoelement RF vs volfrac and areafrac. First ...,3 pixels,volfrac nominal,areafrac nominal,2812 cardinal,NaN
4,5,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,Monoelement RF vs volfrac and areafrac. First ...,3 pixels,volfrac nominal,areafrac nominal,2584 cardinal,NaN
5,6,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,Monoelement RF vs volfrac and areafrac. First ...,3 pixels,volfrac nominal,areafrac nominal,2923 cardinal,NaN
6,7,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,Monoelement RF vs volfrac and areafrac. First ...,3 pixels,volfrac nominal,areafrac nominal,2584 cardinal,NaN
7,8,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,Monoelement RF vs volfrac and areafrac. First ...,5 pixels,volfrac nominal,areafrac nominal,2356 cardinal,NaN
8,9,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,Monoelement RF vs volfrac and areafrac. First ...,5 pixels,volfrac nominal,areafrac nominal,2356 cardinal,NaN
9,10,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,Monoelement RF vs volfrac and areafrac. First ...,5 pixels,volfrac nominal,areafrac nominal,2356 cardinal,NaN


## Load in the dataset measurement table

In a dataset from 1 to n samples can be present.

We save them in the dataset_measurements table.

We have to locate the measurement that contains the samples present in this dataset to link it.

In [91]:
measurement_data = qrs.get_data_metadata('measurements')

measurement_ids = []

for path in file_paths:

    measurement_ids.append(measurement_data[measurement_data['file_path_measurement'] == str(path)]['id_measurement'].values[0])

assert len(measurement_ids) == len(file_paths), 'The number of measurements is not the same as the number of file paths'

In [92]:
relational_table_name = 'dataset_measurements'

for measurement_id in measurement_ids:

    relational_parameters = {'dataset_id': row_id, 'measurement_id': measurement_id}

    # Extract column names and values from the attributes dictionary
    columns = ', '.join(relational_parameters.keys())
    values = ', '.join([f"'{v}'" for v in relational_parameters.values()])

    # Construct the SQL INSERT statement
    sql = f"INSERT INTO {relational_table_name} ({columns}) VALUES ({values})"

    print(sql)

    # Create a cursor object using the cursor() method
    cursor = conn.cursor()

    # Execute the SQL statement
    cursor.execute(sql)

    cursor.execute('COMMIT')

    cursor.close()

INSERT INTO dataset_measurements (dataset_id, measurement_id) VALUES ('64', '34')
INSERT INTO dataset_measurements (dataset_id, measurement_id) VALUES ('64', '11')


## Make sure insert was correct

In [93]:
data = qrs.relation_metadata(table_name,'measurements',relational_table_name)

data

,id_dataset,file_path_dataset,description_dataset,patch_size_dataset,target_dataset,target2_dataset,rows_dataset,shape_dataset,id_measurement,file_path_measurement,parent_measurement_id_measurement,measurementtype_id_measurement,height_measurement,width_measurement,depth_measurement,dtype_measurement,file_type_measurement,signal_type_measurement,aligned_measurement
0,1,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,Monoelement RF vs volfrac and areafrac. First ...,3 pixels,volfrac nominal,areafrac nominal,2686 cardinal,NaN,3,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,1.0,3,505 cardinal,120 cardinal,45 cardinal,uint8 nominal,tif nominal,RF nominal,NaN
1,1,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,Monoelement RF vs volfrac and areafrac. First ...,3 pixels,volfrac nominal,areafrac nominal,2686 cardinal,NaN,41,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,26.0,2,225 cardinal,3279 cardinal,1542 cardinal,uint8 nominal,folder nominal,NaN,True boolean
2,2,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,Monoelement RF vs volfrac and areafrac. First ...,3 pixels,volfrac nominal,areafrac nominal,2686 cardinal,NaN,4,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,1.0,3,505 cardinal,120 cardinal,49 cardinal,uint8 nominal,tif nominal,RF nominal,NaN
3,2,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,Monoelement RF vs volfrac and areafrac. First ...,3 pixels,volfrac nominal,areafrac nominal,2686 cardinal,NaN,40,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,27.0,2,213 cardinal,3271 cardinal,1559 cardinal,uint8 nominal,folder nominal,NaN,True boolean
4,3,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,Monoelement RF vs volfrac and areafrac. First ...,3 pixels,volfrac nominal,areafrac nominal,2923 cardinal,NaN,5,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,1.0,3,505 cardinal,120 cardinal,48 cardinal,uint8 nominal,tif nominal,RF nominal,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,62,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,Monoelement RF vs volfrac and areafrac. First ...,9 pixels,volfrac nominal,areafrac nominal,1792 cardinal,NaN,9,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,7.0,3,505 cardinal,117 cardinal,49 cardinal,uint8 nominal,tif nominal,RF nominal,NaN
108,63,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,Monoelement RF vs volfrac and areafrac. First ...,9 pixels,volfrac nominal,areafrac nominal,2044 cardinal,NaN,35,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,32.0,2,216 cardinal,3308 cardinal,1776 cardinal,uint8 nominal,folder nominal,NaN,True boolean
109,63,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,Monoelement RF vs volfrac and areafrac. First ...,9 pixels,volfrac nominal,areafrac nominal,2044 cardinal,NaN,10,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,7.0,3,505 cardinal,117 cardinal,51 cardinal,uint8 nominal,tif nominal,RF nominal,NaN
110,64,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,Monoelement RF vs volfrac and areafrac. First ...,9 pixels,volfrac nominal,areafrac nominal,1792 cardinal,NaN,34,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,33.0,2,219 cardinal,3233 cardinal,1620 cardinal,uint8 nominal,folder nominal,NaN,True boolean
